**Tools**

*Key concepts*

* Tools are a way to encapsulate a function and its schema in a way that can be passed to a chat model.
* Create tools using the @tool decorator, which simplifies the process of tool creation, supporting the following:
    1. Automatically infer the tool's name, description and expected arguments, while also supporting customization.
    2. Defining tools that return artifacts (e.g. images, dataframes, etc.)
    3. Hiding input arguments from the schema (and hence from the model) using injected tool arguments

*Create tools using the @tool decorator*

The recommended way to create tools is using the @tool decorator. This decorator is designed to simplify the process of tool creation and should be used in most cases. After defining a function, you can decorate it with @tool to create a tool that implements the Tool Interface.



In [2]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
   """Multiply two numbers."""
   return a * b

LangChain has a few other ways to create tools; e.g., by sub-classing the BaseTool class or by using StructuredTool. These methods are shown in the how to create custom tools guide, but we generally recommend using the @tool decorator for most cases.

*Use the tool directly*

In [5]:
response = multiply.invoke({"a": 2, "b": 3})
response

6

Inspect

In [6]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


*Configuring the schema*

The @tool decorator offers additional options to configure the schema of the tool (e.g., modify name, description or parse the function's doc-string to infer the schema).

Please see the API reference for @tool for more details and review the how to create custom tools guide for examples.

*Tool artifacts*

There are artifacts of a tool's execution that we want to make accessible to downstream components in our chain or agent, but that we don't want to expose to the model itself. For example if a tool returns a custom object, a dataframe or an image, we may want to pass some metadata about this output to the model without passing the actual output to the model. At the same time, we may want to be able to access this full output elsewhere, for example in downstream tools.

In [ ]:
@tool(response_format="content_and_artifact")
def some_tool(...) -> Tuple[str, Any]:
    """Tool that does something."""
    ...
    return 'Message for chat model', some_artifact 

*Special type annotations*

The following type annotations will end up removing the argument from the tool's schema. This can be useful for arguments that should not be exposed to the model and that the model should not be able to control.

1. InjectedToolArg: Value should be injected manually at runtime using .invoke or .ainvoke.
2. RunnableConfig: Pass in the RunnableConfig object to the tool.
3. InjectedState: Pass in the overall state of the LangGraph graph to the tool.
4. InjectedStore: Pass in the LangGraph store object to the tool.

You can also use the Annotated type with a string literal to provide a description for the corresponding argument that WILL be exposed in the tool's schema.

    Annotated[..., "string literal"] -- Adds a description to the argument that will be exposed in the tool's schema.

*InjectedToolArg*

There are cases where certain arguments need to be passed to a tool at runtime but should not be generated by the model itself. For this, we use the InjectedToolArg annotation, which allows certain parameters to be hidden from the tool's schema.

For example, if a tool requires a use

In [ ]:
from langchain_core.tools import tool, InjectedToolArg

@tool
def user_specific_tool(input_data: str, user_id: InjectedToolArg) -> str:
    """Tool that processes input data."""
    return f"User {user_id} processed {input_data}"

*RunnableConfig*

You can use the RunnableConfig object to pass custom run time values to tools.

The config will not be part of the tool's schema and will be injected at runtime with appropriate values.

In [ ]:
from langchain_core.runnables import RunnableConfig

@tool
async def some_func(..., config: RunnableConfig) -> ...:
    """Tool that does something."""
    # do something with config
    ...

await some_func.ainvoke(..., config={"configurable": {"value": "some_value"}})

*Best practices*

When designing tools to be used by models, keep the following in mind:

* Tools that are well-named, correctly-documented and properly type-hinted are easier for models to use.
* Design simple and narrowly scoped tools, as they are easier for models to use correctly.
* Use chat models that support tool-calling APIs to take advantage of tools.

*ToolKits*

LangChain has a concept of toolkits. This a very thin abstraction that groups tools together that are designed to be used together for specific tasks.

All Toolkits expose a get_tools method which returns a list of tools. You can therefore do:

In [ ]:
# Initialize a toolkit
toolkit = ExampleTookit(...)

# Get list of tools
tools = toolkit.get_tools()